# Calculating Climatologies

## Overview
Calculating climatologies in Python is a common task in geoscience workflows. This notebook will cover:

- Working with [`xarray`](https://docs.xarray.dev/en/stable/) and its [`groupby()`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby.html) functionality
- A resource guide to point you to more detailed information depending on your use case

---

## Example Data

The dataset used in this notebook originated from the Community Earth System Model v2 (CESM2), and is retrieved from the [Pythia-datasets repository](https://github.com/ProjectPythia/pythia-datasets/tree/main)

The dataset contains 15 years of monthly mean sea surface temperatures (TOS) from January 2000 to December 2014

In [ ]:
from pythia_datasets import DATASETS
import xarray as xr

# Get data
filepath = DATASETS.fetch("CESM2_sst_data.nc")
ds = xr.open_dataset(filepath)

ds

## Calculating Anomalies

You can use the [`groupby()`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.groupby.html) function to group the data by various timescales. From the [xarray user guide](https://xarray.pydata.org/en/v2024.06.0/user-guide/time-series.html#datetime-components):
> xarray also supports a notion of “virtual” or “derived” coordinates for datetime components implemented by pandas, including `year`, `month`, `day`, `hour`, `minute`, `second`, `dayofyear`, `week`, `dayofweek`, `weekday` and `quarter`. For use as a derived coordinate, xarray adds `season` to the list of datetime components supported by pandas.
>


In [ ]:
from pythia_datasets import DATASETS
import xarray as xr
import matplotlib.pyplot as plt

# Get data
filepath = DATASETS.fetch("CESM2_sst_data.nc")
ds = xr.open_dataset(filepath)

# Calculate monthly anomaly
tos_monthly = ds.tos.groupby(ds.time.dt.month)
tos_clim = tos_monthly.mean(dim="time")
tos_anom = tos_monthly - tos_clim

tos_anom

### Visualization

In [ ]:
# Plot the first time slice of the calculated anomalies
tos_anom.isel(time=0).plot()

## Removing Annual Cycle

Also known as [seasonal adjustment](https://en.wikipedia.org/wiki/Seasonal_adjustment) or deseasonalization, it is often used to examine underlying trends in data with a repeating cycle. 

In [ ]:
from pythia_datasets import DATASETS
import xarray as xr

# Get data
filepath = DATASETS.fetch("CESM2_sst_data.nc")
ds = xr.open_dataset(filepath)

# Remove annual cycle from the global monthly mean tos
tos_monthly = ds.tos.groupby(ds.time.dt.month)
tos_clim = tos_monthly.mean(dim="time")
tos_anom = tos_monthly - tos_clim
tos_rmAnnCyc = tos_anom.mean(dim=["lat", "lon"])

tos_rmAnnCyc

### Visualization

In [ ]:
# Plot the global mean tos with the annual cycle removed
tos_rmAnnCyc.plot()
plt.title("Seasonanlly adjusted global mean tos")
plt.ylabel("tos anomaly (°C)")
plt.xlabel("Time (months)")

## Calculating Long Term Means

In [ ]:
from pythia_datasets import DATASETS
import xarray as xr
import matplotlib.pyplot as plt

# Get data
filepath = DATASETS.fetch("CESM2_sst_data.nc")
ds = xr.open_dataset(filepath)

# Calculate long term mean
tos_monthly = ds.tos.groupby(ds.time.dt.month)
tos_clim = tos_monthly.mean(dim="time")

tos_clim

### Visualization

In [ ]:
# Plot an example location of the calculated long term means
tos_clim.sel(lon=310, lat=50, method="nearest").plot()
plt.ylabel("Mean tos (°C)")
plt.xlabel("Month")

## Calculating Seasonal Means

From the [xarray user guide](https://xarray.pydata.org/en/v2024.06.0/user-guide/time-series.html#datetime-components):
>The set of valid seasons consists of `‘DJF’`, `‘MAM’`, `‘JJA’` and `‘SON’`, labeled by the first letters of the corresponding months.
>
If you need to work with custom seasons, the [GeoCAT-comp package](https://geocat-comp.readthedocs.io/en/stable/getting-started.html) offers [`geocat.comp.climatologies.month_to_season()`](https://geocat-comp.readthedocs.io/en/stable/user_api/generated/geocat.comp.climatologies.month_to_season.html) which can be used to create custom three-month seasonal means.

In [ ]:
from pythia_datasets import DATASETS
import xarray as xr
import matplotlib.pyplot as plt

# Get data
filepath = DATASETS.fetch("CESM2_sst_data.nc")
ds = xr.open_dataset(filepath)

# Calculate seasonal means
tos_seasonal = ds.tos.groupby(ds.time.dt.season).mean(dim="time")

tos_seasonal

### Visualization

In [ ]:
# Plot the JJA time slice of the calculated seasonal means
tos_seasonal.sel(season="JJA").plot()

## Finding The Standard Deviations of Monthly Means

Calculate the standard deviations of monthly means for each month using the [`.std()`](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.std.html) function. 

In [ ]:
from pythia_datasets import DATASETS
import xarray as xr
import matplotlib.pyplot as plt

# Get data
filepath = DATASETS.fetch("CESM2_sst_data.nc")
ds = xr.open_dataset(filepath)

# Calculate the standard deviation from monthly mean data
stdMon = ds.tos.groupby(ds.time.dt.month).std()

stdMon

### Visualization

In [ ]:
# Plot the January time slice of the calculated standard deviaitons
stdMon.sel(month=1).plot()

## Curated Resources

To learn more about calculating climatologies in Python, we suggest:

- This [Climatematch Academy notebook](https://comptools.climatematch.io/tutorials/W1D1_ClimateSystemOverview/student/W1D1_Tutorial5.html) on xarray Data Analysis and Climatology
- This [Project Pythia Foundations tutorial](https://foundations.projectpythia.org/core/xarray/computation-masking.html) on Computations and Masks with xarray
- The [xarray user guide](https://docs.xarray.dev/en/stable/user-guide/time-series.html) on working with time series data